This notebook provides a brief tutorial for how to solve fully-commuting Hamiltonians using this package. A fully-commuting Hamiltonian in which all Pauli terms mutually commute:

$$
H = \sum_{P \in \mathcal{S}} h_P P, \qquad \forall P, Q \in \mathcal{S}, \ [P, Q] = 0.
$$

Such a Hamiltonian can be written as a polynomial function of a set of algebraically independent, mutually commuting generators $\overline{C} = \{C_0,\ldots,C_{K-1}\}$. This statement means two things. First: the term "algebraically independent" means that none of the generators $C_k$ can be written as a product, up to a phase, of the other generators $C_l, l\not=k$. Second: each Pauli term in $H$ can be written as a product, up to a phase, of the full set of generators $\overline{C}$:


$$
P = e^{\text{i}\theta_P}\prod_{k=0}^{K-1}C_k^{v_{k,P}}.
$$

where $v_{k,P} \in \{0,1\}$, which follows from the fact that Pauli operators are involutory. Therefore,

$$
H = \sum_{P \in \mathcal{S}} h_P \left[e^{\text{i}\theta_P}\prod_{k=0}^{K-1}C_k^{v_{k,P}}\right] = p(C_0,\ldots,C_{K-1}).
$$

Any fully-commuting Hamiltonian can be diagonalized by a Clifford unitary. Any Clifford unitary can be written in the following way:

$$
U_\text{c} = \prod_{i=1}^{L} \frac{A_i + B_i}{\sqrt{2}}, \qquad \forall i,\ \{A_i, B_i\} = 0,
$$

where $\{X, Y\} = XY + YX$ is the anti-commutator. For a Clifford unitary to diagonalize $H$, it is sufficient to map the independent generators $C_0,\ldots,C_{K-1}$ to single qubit $z$ operators:

$$
U_\text{c}^\dagger C_k U_\text{c} = z_k \implies U_\text{c}^\dagger H U_\text{c} = p(z_0,\ldots,z_{K-1})
$$

The spectral decomposition of $H$, which reveals the eigenvalues and eigenstates of $H$, is:
$$
H = \sum_{\vec{z} \in \{0,1\}^{K}} p(v_0,\ldots,v_{K-1}) U_\text{c}|\vec{z}\rangle\langle\vec{z}| U_\text{c}^\dagger
$$
where $v_k = (-1)^{z_k}$. Note that the same polynomial function which defines $H$ also is used to obtain the eigenvalues of $H$. Therefore, to diagonalize a fully-commuting Hamiltonian, there are three steps:

1. Find a set of algebraically independent, mutually commuting, generators $\overline{C}$ for $H$.
2. Write $H$ as a polynomial function $p$ of the generators.
3. Find the Clifford unitary $U_\text{c}$ that maps the generators to single qubit $z$ operators. 

This is in essence what the code in `utils_fc.py` does.

In [ ]:
import numpy as np

from openfermion import (
    QubitOperator,
    get_sparse_operator
)

from utils_fc import (
    random_fc_hamiltonian,
    solve_fc_hamiltonian
)

from utils_basic import (
    copy_hamiltonian,
    apply_unitary_product
)
from itertools import product

# generate random fully-commuting Hamiltonian, also numerically obtain its eigenvalues to 8 decimal places
# Nqubits -> number of qubits for the fully-commuting Hamiltonian
# Nterms  -> number of terms for the fully-commuting Hamiltonian
# note: if Nterms / Nqubits is too large, the function random_fc_hamiltonian will run forever

Nqubits = 8
Nterms  = 25
H       = random_fc_hamiltonian(Nqubits, Nterms)
eigs    = np.round(np.linalg.eigh(get_sparse_operator(H, Nqubits).toarray())[0], 8)

# diagonalize H
#     the solve_fc_hamiltonian function returns three things
#         1. a list of QubitOperator C, which are the generators of H
#         2. a function p, which takes as input C to return H, or a list of {-1,1} to return eigenvalues of H
#         3. a list of QubitOperator Uc. Each entry of Uc is of the form 1/np.sqrt(2) (A + B), where A and B 
#            anti-commute. Uc is the diagonalizing unitary for H

p, C, Uc  = solve_fc_hamiltonian(H, Nqubits)
Ngen      = len(C)

# Here, we check if some facts relating H, and (p,C) are true

# verification 1: H should be equal to the polynomial of the generators

assert H - p(C) == QubitOperator().zero()
print("H = p(C): passed")

# verification 2: for all lists v of {-1,1}, p(v) should be an eigenvalue of H

for v in product([-1,1], repeat=Ngen):
    v  = np.array(v)
    assert np.round(p(v), 8) in eigs
print("p(v) are eigenvalues of H: passed")

# now we look at the Clifford unitary
# to apply a Clifford unitary to a QubitOperator efficiently, use the `apply_unitary_product` function
# note: this is not the most efficient route to implementing Clifford unitaries, but it is sufficient for 
# small systems

# verification 3: Uc^\dagger C[k] U_c = z_k

for k in range(Ngen):
    assert apply_unitary_product(C[k], Uc) - QubitOperator(f'Z{k}') == QubitOperator().zero()
print("U_c^\\dagger C[k] U_c = z_k: passed")

# verification 4: U_c^\dagger H U_c = p(z_0, \ldots, z_{K-1})

rotated_H        = apply_unitary_product(H, Uc)
ising_generators = [QubitOperator(f'Z{k}') for k in range(Ngen)]
H_ising          = p(ising_generators)

assert rotated_H - H_ising == QubitOperator().zero()

print('U_c^\\dagger H U_c = p(z_0,\\ldots,z_{K-1}): passed')

H = p(C): passed
p(v) are eigenvalues of H: passed
U_c^\dagger C[k] U_c = z_k: passed
U_c^\dagger H U_c = p(z_0,\ldots,z_{K-1}): passed
